# Contents:

#### Importing libraries and data frames
#### Creating a region flag
#### Crossing the region flag with the spending flag
#### Creating an exclusion flag for customers with under 5 orders
#### Creating customer profiling variables (family status, vegan status, and age categories)
#### Grouping and aggregation (order frequency and expenditure)
#### Visualizations
#### Crosstabulation and aggregation of customer profiling variables
#### Saving and exporting to a pkl file

## Importing libraries and data frames

In [8]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

In [9]:
path = r'C:\Users\emman\OneDrive\Documents\Instacart Basket Analysis (Oct-22)'

In [10]:
ord_prods_cust_merged = pd.read_pickle(os.path.join(path, '02 Data', 'prepared Data', 'ord_prods_cust_merged.pkl'))

In [11]:
df = ord_prods_cust_merged

In [12]:
# To drop unneeded _merge column
df.drop(columns={'_merge'},inplace=True)

### Q2. Consider any security implications that might exist for this new data. You’ll need to address any PII data in the data before continuing your analysis.

 - **Reflection**: customer data set has a lot of PII data, including names, address and salary. These PII data will not be removed as it is not specified in the project brief. However, I will make a note of it in the final report.

### Q3. The Instacart officers are interested in comparing customer behavior in different geographic areas. Create a regional segmentation of the data. You’ll need to create a “Region” column based on the “State” column from your customers data set.

 - Use the region information in this Wikipedia article to create your column (you only need to create regions, not divisions).
 - Determine whether there’s a difference in spending habits between the different U.S. regions. (Hint: You can do this by crossing the variable you just created with the spending flag.)

## Creating a region flag

In [13]:
# defining regions as list objects
northeast=['Maine','New Hampshire','Vermont','Massachusetts','Rhode Island','Connecticut','New York','Pennsylvania','New Jersey']
midwest=['Wisconsin','Michigan','Illinois','Indiana','Ohio','North Dakota','South Dakota','Nebraska','Kansas','Minnesota','Iowa','Missouri']
south=['Delaware','Maryland','District of Columbia','Virginia','West Virginia','North Carolina','South Carolina','Georgia','Florida','Kentucky','Tennessee','Mississippi','Alabama','Oklahoma','Texas','Arkansas','Louisiana']
west=['Idaho','Montana','Wyoming','Nevada','Utah','Colorado','Arizona','New Mexico','Alaska','Washington','Oregon','California','Hawaii']

In [14]:
# creating an empty list and for loop logic
result=[]

for value in df['state']:
    if value in northeast:
        result.append('Northeast')
    elif value in midwest:
        result.append('Midwest')
    elif value in south:
        result.append('South')
    elif value in west:
        result.append('West')

In [15]:
# creating the region column based on the for loop results
df['regions']=result

In [16]:
# To check the value counts in the regions column
df['regions'].value_counts()

South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: regions, dtype: int64

In [17]:
# To check that the regions column is the same length as the rest of the data frame
len(df['regions'])

32404859

In [18]:
df.head(5)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,first_name,last_name,gender,state,age,date_joined,no_of_dependents,marital_status,income,regions
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,South
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,South
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,South
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,South
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,South


#### Determine whether there’s a difference in spending habits between the different U.S. regions. (Hint: You can do this by crossing the variable you just created with the spending flag.) 

In [21]:
# Create a flag that assigns a “types of spenders” label to a user ID based on average prices.

df.loc[df['avg_price'] < 10, 'spender_flag'] = 'Low spender'

In [22]:
df.loc[df['avg_price'] >= 10, 'spender_flag'] = 'High spender'

In [23]:
df['spender_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spender_flag, dtype: int64

In [26]:
# Creating a crosstab to determine difference in spending power across regions
crosstab=pd.crosstab(df['regions'],df['spender_flag'],dropna=False)

In [27]:
pd.crosstab(index=df.regions, columns=df.spender_flag)

spender_flag,High spender,Low spender
regions,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


#### Each region has very similar proportions of high vs. low spenders (around 33% high spenders and 67% low spenders)

### Q4. The Instacart CFO isn’t interested in customers who don’t generate much revenue for the app. Create an exclusion flag for low-activity customers (customers with less than 5 orders) and exclude them from the data. Make sure you export this sample.

In [29]:
# To create an activity flag based on max_order
df.loc[df['max_order']>=5,'activity']='normal'
df.loc[df['max_order']<5,'activity']='low'

In [30]:
# To check the value counts in the activity flag column
df['activity'].value_counts()

normal    30964564
low        1440295
Name: activity, dtype: int64

In [31]:
# To check that the length of the activity column matches the dataframe
len(df['activity'])

32404859

In [32]:
# To create a dataframe containing only low activity customers
df_low=df[df['activity']=='low']

In [33]:
df_low.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,last_name,gender,state,age,date_joined,no_of_dependents,marital_status,income,regions,activity
1510,520620,120,1,3,11,NaN,196,2,0,Soda,...,Rich,Female,Kentucky,54,3/2/2017,2,married,99219,South,low
1511,3273029,120,3,2,8,19.0,196,2,1,Soda,...,Rich,Female,Kentucky,54,3/2/2017,2,married,99219,South,low
1512,520620,120,1,3,11,NaN,46149,1,0,Zero Calorie Cola,...,Rich,Female,Kentucky,54,3/2/2017,2,married,99219,South,low
1513,3273029,120,3,2,8,19.0,46149,1,1,Zero Calorie Cola,...,Rich,Female,Kentucky,54,3/2/2017,2,married,99219,South,low
1514,520620,120,1,3,11,NaN,26348,3,0,Mixed Fruit Fruit Snacks,...,Rich,Female,Kentucky,54,3/2/2017,2,married,99219,South,low


In [34]:
df_low.shape

(1440295, 35)

In [ ]:
# Exporting low activity sample as a pkl file
df_low.to_pickle(os.path.join(path,'Data','Prepared Data','low_activity_user_data.pkl'))